<a href="https://colab.research.google.com/github/leabizbille/ProjetIOT/blob/main/Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install summarytools -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00


In [2]:
!pip install pygwalker -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 956.2/956.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 36.8 MB/s eta 0:00:00


In [3]:
pip install sweetviz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 42.5 MB/s eta 0:00:00


In [4]:
import pygwalker as pyg
from summarytools import dfSummary # module pandas-profiling
import pandas as pd
import sweetviz as sv # Module pour faire le rapport HTML
import missingno as msno

In [5]:
# Charger le fichier CSV en conservant les titres des colonnes
val_roland = pd.read_csv("/content/Val de Roland_2024.csv",sep= ";", parse_dates=[0], dayfirst=True)
#THabitat2023 = pd.read_csv("/content/Tours Habitat_2023.csv",sep= ";", parse_dates=[0], dayfirst=True)
#THabitat2024= pd.read_csv("/content/Tours Habitat_2024.csv",sep= ";", parse_dates=[0], dayfirst=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Val de Roland_2024.csv'

In [ ]:
msno.matrix(val_roland)

In [ ]:
# Description pour comprendre les données et mettre en suite le tableau en forme
dfSummary(val_roland)

- 49 % de nan dans les Unités -> regarder pour faire des modifications
- 99.5 % de donnée manquante dans "Type Générique". Chercher l'utilisation et analyse l'interet.
- Le logement 2 à 43% des données. Soit 3 x plus que le logement 3 et 4, chacun.

Pour toutes les colonnes, reformater les données.
Un pivotement ou une transposition du tableau sera probablement nécessaire.

Mélange des modalites entre appartement et type d équipement

In [ ]:
# Séparer la colonne 'datetime' en deux colonnes distinctes
val_roland[['date', 'time']] = val_roland['Date'].str.split(' ', expand=True)

# Convertir les nouvelles colonnes au format datetime
val_roland['date'] = pd.to_datetime(val_roland['date'], format='%Y-%m-%d')
val_roland['time'] = pd.to_datetime(val_roland['time'], format='%H:%M:%S').dt.time

# Supprimer l'ancienne colonne si elle n'est plus nécessaire
val_roland= val_roland.drop(columns=['Date'])

In [ ]:
# Extraction des informations dans de nouvelles colonnes
val_roland['Logement'] = val_roland['Objet'].str.extract(r'(LOGEMENT\d+)')
val_roland['Replicat'] = val_roland['Objet'].str.extract(r'-\s([AB])')
val_roland['Pilote'] = val_roland['Objet'].str.contains(r'\(piloté\)').map({True: 'piloté', False: 'non piloté'})

# Supprimer l'ancienne colonne
val_roland= val_roland.drop(columns=['Objet'])

In [ ]:
# Trouver toutes les modalités uniques dans la colonne 'Équipement'
modalites = val_roland['Équipement'].unique()

# Afficher les modalités
print(modalites)

In [ ]:
# Remplacer les valeurs avec "_" par ce qui se trouve après, sinon garder la valeur intacte
val_roland['Équipement'] = val_roland['Équipement'].str.split('_').str[1].fillna(val_roland['Équipement'])
val_roland['Équipement'] = val_roland['Équipement'].str.split(' - ').str[1].fillna(val_roland['Équipement'])


In [ ]:
# Fonction de nettoyage
def nettoyer_equipement(data):
    data = data.strip()  # Supprime les espaces au début et à la fin
    if data.startswith('WIFITIC'):  # Vérifie si le texte commence par "WIFITIC"
        return 'Conso_Total_index'
    if data.startswith('Thermostat'):
        return 'Thermostat'
    if data.startswith('TEMP'): # Changement de nom et clarifie
        return 'Thermostat'
    elif data == 'Conso ':  # Nettoie "Conso " avec un espace
        return 'Conso'
    elif data == 'Conso': # Changement de nom
        return 'Conso_horaire'
    elif data == 'occupation':  # Clarifie
        return 'Occupation_Aujourdhui'
    elif data == 'arrivée du jour':  # Clarifie
        return 'Occupation_Hier'
    else:
        return data  # Conserve les autres modalités intactes

# Appliquer la fonction de nettoyage à la colonne Equipement
val_roland['Équipement'] = val_roland['Équipement'].apply(nettoyer_equipement)

In [ ]:
# Trouver toutes les modalités uniques dans la colonne 'Équipement'
modalites = val_roland['Équipement'].unique()

# Afficher les modalités
print(modalites)

* Attention au 4 PM qui correspond à beaucoup de chose !

In [ ]:
# Trouver toutes les modalités uniques dans la colonne 'Équipement'
modalites = val_roland['Commande'].unique()

# Afficher les modalités
print(modalites)

In [ ]:
# Fonction de nettoyage
def nettoyer_commande(data):
    data = data.strip()  # Supprime les espaces au début et à la fin
    if data.startswith('Conso_Heure'):  # Vérifie si le texte commence par Conso_Heure
        return 'Conso_Total_kw/h'
    if data.startswith('0 - CHAUFFAGE Etat'):
        return 'Chauffage_OnOff'
    if data.startswith('0 - CHAUFFAGE instantané'): # Changement de nom et clarifie
        return 'Chauffage_Conso'
    elif data == '0 - CHAUFFAGE total':  # Nettoie "Conso " avec un espace
        return 'Chauffage_Conso_Total'
    elif data == '1 - ECS Etat': # Changement de nom
        return 'Ballon_OnOff'
    elif data == '2 - Prise Etat': # Changement de nom
        return 'TV-Frigo_OnOff'
    elif data == 'Consigne': # Changement de nom
        return 'TemperatureDesiree'
    elif data == 'Température': # Changement de nom
        return 'TemperatureReelle'
    elif data == 'index': # Changement de nom
        return 'TIC_enedis'
    elif data == 'Température': # Changement de nom
        return 'TemperatureReelle'
    elif data == 'Sw 1 Etat': # Changement de nom
        return 'Ballon_OnOff'
    elif data == 'Sw 2 Etat': # Changement de nom
        return 'TV-Frigo_OnOff'
    elif data == 'Etat occupation':  # Clarifie
        return 'Presence_absence'
    elif data == 'Etat arrivée du jour':  # Clarifie
        return 'Presence_absence'
    else:
        return data  # Conserve les autres modalités intactes

# Appliquer la fonction de nettoyage à la colonne Equipement
val_roland['Commande'] = val_roland['Commande'].apply(nettoyer_commande)

In [ ]:
# Trouver toutes les modalités uniques dans la colonne 'Équipement'
modalites = val_roland['Commande'].unique()

# Afficher les modalités
print(modalites)

Mettre "Index" comme unité, si TIC enedis dans la colonne commande.

In [ ]:
# Mettre "index" dans la colonne 'Unité' si "index" est dans la colonne 'Commande', sinon garder la valeur existante
val_roland['Unité'] = val_roland.apply(lambda row: 'Oui-Non' if 'Presence_absence' in row['Commande'] and pd.isna(row['Unité']) else row['Unité'], axis=1)

Mettre "ON-Off" comme unité, si la ligne correspond à un object Allumé ou éteint.

In [ ]:
# Mettre "On-Off" dans la colonne 'Unité' si " ... " est dans la colonne 'Commande', sinon garder la valeur existante
val_roland['Unité'] = val_roland.apply(lambda row: 'On-Off' if 'Chauffage_OnOff' in row['Commande'] and pd.isna(row['Unité']) else row['Unité'], axis=1)
val_roland['Unité'] = val_roland.apply(lambda row: 'On-Off' if 'Ballon_OnOff' in row['Commande'] and pd.isna(row['Unité']) else row['Unité'], axis=1)
val_roland['Unité'] = val_roland.apply(lambda row: 'On-Off' if 'TV-Frigo_OnOff' in row['Commande'] and pd.isna(row['Unité']) else row['Unité'], axis=1)

In [ ]:
# Mettre "Oui-Non" dans la colonne 'Unité' si "Presence humaine" est dans la colonne 'Commande', sinon garder la valeur existante
val_roland['Unité'] = val_roland.apply(lambda row: 'On-Off' if 'Chauffage_OnOff' in row['Commande'] and pd.isna(row['Unité']) else row['Unité'], axis=1)

Verification du nombre de valeur NA

In [ ]:
print(val_roland.isna().sum())

In [ ]:
msno.matrix(val_roland)

In [ ]:
# Description pour comprendre les données et mettre en suite le tableau en forme
dfSummary(val_roland)

In [ ]:
# Mettre "Chauffage" si les conditions sont remplies
val_roland['Équipement'] = val_roland.apply(
    lambda row: 'Chauffage' if 'Chauffage_OnOff' in row['Commande']
    and row['Équipement'] == '4PM' else row['Équipement'], axis=1)

val_roland['Équipement'] = val_roland.apply(
    lambda row: 'Chauffage' if 'Chauffage_Conso' in row['Commande']
    and row['Équipement'] == '4PM' else row['Équipement'], axis=1)

val_roland['Équipement'] = val_roland.apply(
    lambda row: 'Chauffage' if 'Chauffage_Conso_Total' in row['Commande']
    and row['Équipement'] == '4PM' else row['Équipement'], axis=1)

# Mettre "Ballon_Eau" si les conditions sont remplies
val_roland['Équipement'] = val_roland.apply(
    lambda row: 'Ballon_Eau' if 'Ballon_OnOff' in row['Commande']
    and row['Équipement'] == '4PM' else row['Équipement'], axis=1)

# Mettre "TV-Frigo" si les conditions sont remplies
val_roland['Équipement'] = val_roland.apply(
    lambda row: 'TV-Frigo' if 'TV-Frigo_OnOff' in row['Commande']
    and row['Équipement'] == '4PM' else row['Équipement'], axis=1)


Rapport des données nettoyées :

In [ ]:
my_report = sv.analyze(val_roland)

In [ ]:
my_report.show_html()

Rapport des données de Val Roland, binaire entre les appartements Piloté et les appartements non pilotés.

In [ ]:
my_report = sv.compare_intra(val_roland, val_roland["Pilote"] == "non piloté", ["Non piloté", "Piloté"])

In [ ]:
my_report.show_html()

In [ ]:
# Vérifier les types de données pour validation
print(val_roland.info())

In [ ]:
val_roland['Valeur'] = pd.to_numeric(val_roland['Valeur'].str.replace(',', '.'), errors='coerce')